# Notebook 1: Creating Items for Building Complexes

This notebook implements the first step of creating of the Klosterdatenbank-to-Factgrid-Workflow which is to create Items for the building complexes. It contains describing elements about the underlying data model and the workflow in general, as well as specific instructions in order to run the notebook. Markdown cells containing describing elements are marked as `#description`. Instructional sections are marked as `#instruction`.

Strictly speaking, the monastery database does not contain dedicated information on building complexes. Information on where a religious community had its place of operation is stored in the `gs_monastery_location` table. This table assigns each row of a religious community (`gsn_id`) to a location (`place_id`) and, if known, specific coordinates within this location (`longitude`, `latitude`). Such an assignment implies that the community lived or worked at this location at a certain point in time. At this point, we make the central assumption that a building complex of some kind, consisting of at least one building, must have existed. Accordingly, the building complexes created in this step represent both a row from the `gs_monastery_location` table and thus an assignment of a monastery to a specific location, as well as physical buildings in which religious communities worked and which may have continued to exist before or after their use and have experienced other use scenarios.

The notebook requires the following libraries to run. If an error occurs, make sure the libraries are installed on your system.

In [83]:
import pandas as pd
import numpy as np
import os
import csv

First, the export files are loaded into [Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataframes are stored in a dictionary with the keys being the filenames, for easier access.

In [84]:
# Load Access exports
export_files = {}
for export_file in os.listdir("data/exports_monasteryDB"):
    if export_file.endswith("xlsx"):
        export_files[export_file.split(".")[0]] = f"data/exports_monasteryDB/{export_file}"

# Create dataframes for each table
dataframes = {key: pd.read_excel(value) for key, value in export_files.items()}

# Add dataframe for monasteries in factGrid (stored in a different directory)
dataframes["building_complexes_in_factgrid"] = pd.read_csv("data/factgrid_data/building_complexes_in_factgrid.csv")
dataframes["monasteries_in_factgrid"] = pd.read_csv("data/factgrid_data/monasteries_in_factgrid.csv")

Since `gs_monastery_location` does not contain the name of the monasteries, the table is joined to `gs_monastery` to extract the missing information. The resulting table is cut down to the relevant columns. The resulting dataframe is being filtered to only contain information on religious comunities that have the status "online", meaning they are not currently worked on anymore. Finally, to make sure that no duplicate building complexes are being created, the table is filtered against the existing building complexes in FactGrid.

In [85]:
# Merge gs_monastery_location and gs_monastery
merged_df = pd.merge(dataframes["gs_monastery_location"], dataframes["gs_monastery"], left_on='gsn_id', right_on='id_gsn', how='left')
# Filter for status 'online'
online_df = merged_df[merged_df["status"] == "Online"]
# Define columns to drop
drop_columns = [
    "relocated", 
    "comment", 
    "main_location", 
    "diocese_id", 
    "id_monastery", 
    "date_created", 
    "created_by_user", 
    "note", 
    "patrocinium",
    "selection", 
    "processing_status", 
    "gs_persons", 
    "selection_criteria", 
    "last_change", 
    "changed_by_user", 
    "founder",
    "Unnamed: 0_x",
    "Unnamed: 0_y"
]
# Prepare dataframe by dropping unnecessary columns
prepared_df = online_df.drop(drop_columns, axis="columns")
prepared_df = prepared_df[~prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"])]
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name
0,16673,46484371,10880,1773,NaN,NaN,1869,NaN,NaN,14.067180,49.961381,Beraun,10880,Online,"Piaristenkolleg Beraun (Beroun), Tschechien"
1,6072,19993,3593,1135,1145.0,um 1140,1525,NaN,NaN,10.659167,50.828889,NaN,3593,Online,Zisterzienserkloster Georgenthal
2,1865,46479281,3468,1346,NaN,1346,1541,NaN,1541,14.560000,53.426389,Stettin,3468,Online,"Kollegiatstift St. Otto, Stettin (Szczecin), P..."
3,16824,46484486,11549,1295,1305.0,um 1300,1300,1325.0,frühes 14. Jahrhundert,NaN,NaN,Luditz,11549,Online,"Dominikanerinnenkloster Luditz (Žlutice), Tsch..."
4,16923,46479178,11613,1285,1380.0,wahrscheinlich 1285,1824,NaN,NaN,5.232053,50.524050,Huy,11613,Online,"Magdalenerinnenkloster Saint-Quirin, Huy, Belgien"
5,16936,6305,50228,1478,1478.0,1478/1479,1802,NaN,NaN,6.958232,50.925171,NaN,50228,Online,"Franziskanerterziarinnen St. Bonifatius, Köln"
6,10564,16731,302,1186,NaN,1186/1191,1192,NaN,NaN,11.042222,51.901389,NaN,302,Online,Prämonstratenserstift Halberstadt
7,13544,46481605,8354,1204,NaN,NaN,1584,1594.0,1584/1594,6.424470,53.376130,NaN,8354,Online,Prämonstratenserinnenstift (Nijenklooster) Klo...
8,765,4845,367,1427,NaN,NaN,1484,NaN,NaN,NaN,NaN,Hemeringen,367,Online,Augustinerchorfrauenstift Egestorf
9,367,39870,367,1293,1303.0,ca. 1298,1427,NaN,NaN,NaN,NaN,Egestorf,367,Online,Augustinerchorfrauenstift Egestorf


To double-check potential duplicates, the following cell finds buildings complexes that are connected to monasteries already existent in FactGrid. If the resulting DataFrame is empty, all building complexes will be linked to newly created monastery items.

In [86]:
existing_monasteries = prepared_df[prepared_df["gsn_id"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"])]
existing_monasteries

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name
17,8445,20353,3823,1320,NaN,NaN,1531,NaN,NaN,10.266809,50.718199,NaN,3823,Online,Zisterzienserkloster Georgenzell


It is expected that items in FactGrid have a label in at least one language. While the FactGrid ID (also referred to as the "Q-Number") uniquely identifies the item, the label serves to capture the name of the item in everyday language. The label is also indexed for text-based search. The naming of the items created in this project follows the following rule:
- For the religious communities, the name from the monastery database is used as the label, for example "Zisterzienserkloster Georgenzell".
- For the building complexes, the labels are constructed according to the following schema: `Gebäudekomplex <monastery_name> [(<location_name>)]`. Here, `monastery_name` is again the name of the religious community from the `gs_monastery` table. `location_name` is a column of the `gs_monastery_location` table. In this column, if available, the specific name given to this location is stored. 

For example, the "Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien" (GSN [11665](https://klosterdatenbank.adw-goe.de/gsn/11665)) has two locations in the Belgian town of Sint-Truiden, namely the location "Sint Truiden" and the location "Metsteren" (see Figure). The constructed labels are then "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Sint-Truiden)" and "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Metsteren)". However, location names are not available in all these cases, which can lead to duplicates in the labels. These are displayed in the workflow, so that location names can be added to distinguish them better.

<img src="documentation-images/Standorte GSN11665.png" alt="Monastery Locations of GSN 11665" width="500">

*Figure 1: Building Complexes of the Benedictine nun's monastery Mielen in Sint-Truiden, Belgium (GSN 11665). Base-Layer: OpenStreetMap.*

The following cell constructs the location names and saves them in a new column called "Lde" (see [Quickstatements specification](https://www.wikidata.org/wiki/Help:QuickStatements#Adding_labels,_aliases,_descriptions_and_sitelinks)).

In [87]:
# 1. Create new column with labels
prepared_df['Lde'] = "Gebäudekomplex " + prepared_df["monastery_name"].str.cat(prepared_df["location_name"].fillna(''), sep=" (") +")"
# 2. If necessary, delete empty brackets at end of labels
prepared_df['Lde'] = prepared_df["Lde"].str.replace(r'\(\)', '', regex=True).apply(lambda x: f'\"{x.strip()}\"')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde
0,16673,46484371,10880,1773,NaN,NaN,1869,NaN,NaN,14.067180,49.961381,Beraun,10880,Online,"Piaristenkolleg Beraun (Beroun), Tschechien","""Gebäudekomplex Piaristenkolleg Beraun (Beroun..."
1,6072,19993,3593,1135,1145.0,um 1140,1525,NaN,NaN,10.659167,50.828889,NaN,3593,Online,Zisterzienserkloster Georgenthal,"""Gebäudekomplex Zisterzienserkloster Georgenthal"""
2,1865,46479281,3468,1346,NaN,1346,1541,NaN,1541,14.560000,53.426389,Stettin,3468,Online,"Kollegiatstift St. Otto, Stettin (Szczecin), P...","""Gebäudekomplex Kollegiatstift St. Otto, Stett..."
3,16824,46484486,11549,1295,1305.0,um 1300,1300,1325.0,frühes 14. Jahrhundert,NaN,NaN,Luditz,11549,Online,"Dominikanerinnenkloster Luditz (Žlutice), Tsch...","""Gebäudekomplex Dominikanerinnenkloster Luditz..."
4,16923,46479178,11613,1285,1380.0,wahrscheinlich 1285,1824,NaN,NaN,5.232053,50.524050,Huy,11613,Online,"Magdalenerinnenkloster Saint-Quirin, Huy, Belgien","""Gebäudekomplex Magdalenerinnenkloster Saint-Q..."
5,16936,6305,50228,1478,1478.0,1478/1479,1802,NaN,NaN,6.958232,50.925171,NaN,50228,Online,"Franziskanerterziarinnen St. Bonifatius, Köln","""Gebäudekomplex Franziskanerterziarinnen St. B..."
6,10564,16731,302,1186,NaN,1186/1191,1192,NaN,NaN,11.042222,51.901389,NaN,302,Online,Prämonstratenserstift Halberstadt,"""Gebäudekomplex Prämonstratenserstift Halberst..."
7,13544,46481605,8354,1204,NaN,NaN,1584,1594.0,1584/1594,6.424470,53.376130,NaN,8354,Online,Prämonstratenserinnenstift (Nijenklooster) Klo...,"""Gebäudekomplex Prämonstratenserinnenstift (Ni..."
8,765,4845,367,1427,NaN,NaN,1484,NaN,NaN,NaN,NaN,Hemeringen,367,Online,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges..."
9,367,39870,367,1293,1303.0,ca. 1298,1427,NaN,NaN,NaN,NaN,Egestorf,367,Online,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges..."


As mentioned above, there might be duplicate labels in cases where locations don't have an explicit name. Since they still can be distinguished from another by their identifier and coordinates, this is not necessarily a problem. However, the following cell will create a list of all the duplicate labels so that they can be examined.

**In order to resolve the duplicates**

1. Open and inspect the table located at `data/intermediate_results/duplicate_building_complex_labels.xslx`
2. Add location names in the monastery database
3. Create new exports from the monastery database and replace `data/exports_monasteryDB/gs_monastery.xlsx` and `data/exports_monasteryDB/gs_monastery_location.xlsx` with the new files
4. Re-run the notebook. The cell below now should no longer contain the duplicates you resolved. 

In [88]:
duplicated_building_complex_labels = prepared_df[prepared_df.duplicated(subset="Lde", keep=False)]
duplicated_building_complex_labels.to_excel('data/intermediate_results/duplicate_building_complex_labels.xlsx')
duplicated_building_complex_labels

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde
5,16936,6305,50228,1478,1478.0,1478/1479,1802,NaN,NaN,6.958232,50.925171,NaN,50228,Online,"Franziskanerterziarinnen St. Bonifatius, Köln","""Gebäudekomplex Franziskanerterziarinnen St. B..."
9,367,39870,367,1293,1303.0,ca. 1298,1427,NaN,NaN,NaN,NaN,Egestorf,367,Online,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges..."
14,7909,39870,367,1484,NaN,NaN,1559,NaN,NaN,NaN,NaN,Egestorf,367,Online,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges..."
15,7150,6305,50228,1367,1293.0,Ende des 13. Jahrhunderts,1478,1479.0,1478/1479,6.944923,50.938611,NaN,50228,Online,"Franziskanerterziarinnen St. Bonifatius, Köln","""Gebäudekomplex Franziskanerterziarinnen St. B..."


FactGrid is a multilingual platform. Therefore, the labels for the monasteries and building complexes should not only be created in German, but also in English. Due to the heterogeneity of the monastery names in the database, a rule-based translation is difficult to implement. Instead, a Large-Language Model was used. The model, prompting, and details of the translation are described in more detail in the notebook "1a - Translation". We are using the [GWDG/KISSKI API](https://docs.hpc.gwdg.de/services/chat-ai/index.html), so in order to execute the notebook, a [SAIA API key](https://docs.hpc.gwdg.de/services/saia/index.html) is needed. Since the translation process can take some time, it has been outsourced to a separate notebook.

In [89]:
to_translate = prepared_df[["monastery_name", 'Lde']].copy()
to_translate["note"] = "x"
to_translate["Dde"] = "x"
to_translate = to_translate.rename(columns={"Lde": "building_Lde", "Dde": "building_Dde", "monastery_name" : "monastery_Lde", "note": "monastery_Dde"})
to_translate.to_csv("data/translation/to_translate.csv")
to_translate

,monastery_Lde,building_Lde,monastery_Dde,building_Dde
0,"Piaristenkolleg Beraun (Beroun), Tschechien","""Gebäudekomplex Piaristenkolleg Beraun (Beroun...",x,x
1,Zisterzienserkloster Georgenthal,"""Gebäudekomplex Zisterzienserkloster Georgenthal""",x,x
2,"Kollegiatstift St. Otto, Stettin (Szczecin), P...","""Gebäudekomplex Kollegiatstift St. Otto, Stett...",x,x
3,"Dominikanerinnenkloster Luditz (Žlutice), Tsch...","""Gebäudekomplex Dominikanerinnenkloster Luditz...",x,x
4,"Magdalenerinnenkloster Saint-Quirin, Huy, Belgien","""Gebäudekomplex Magdalenerinnenkloster Saint-Q...",x,x
5,"Franziskanerterziarinnen St. Bonifatius, Köln","""Gebäudekomplex Franziskanerterziarinnen St. B...",x,x
6,Prämonstratenserstift Halberstadt,"""Gebäudekomplex Prämonstratenserstift Halberst...",x,x
7,Prämonstratenserinnenstift (Nijenklooster) Klo...,"""Gebäudekomplex Prämonstratenserinnenstift (Ni...",x,x
8,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges...",x,x
9,Augustinerchorfrauenstift Egestorf,"""Gebäudekomplex Augustinerchorfrauenstift Eges...",x,x


After executing the above cell, a table is generated in `data/translation` that contains all terms that should be translated: `to_translate.csv`. Execute Notebook 1a. Once the execution is completed, there should be a file named `translated.csv` that contains the translations within the `data/translation` folder. Once the file exists, you can run the next cell to load the translated labels.

In [90]:
translated = pd.read_csv("data/translation/translated.csv")
prepared_df["Len"] = translated["building_Len"].apply(lambda x:f'\"{x}\"')
prepared_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/translation/translated.csv'

Our data model separates religious communities from the building complexes in which they lived and worked. The geocoordinates of a location of a religious community are properties of the building complex in this modeling. In the monastery database, there are two levels of accuracy with which the localization of a monastery location can be performed: coordinates for a monastery location will either represent the exact point where the building was located, or the central point of a place, e.g. a village, in which it was located. It is to be noted that the centroid-based location always only represents an approximation of the centroid of the modern location. In cases where the exact location of the building complex is unknown, the respective item will not be linked to any coordinates. Instead, the coordinates of the place where it is located should be queried. In all other cases, the coordinates are directly linked to the building complexes, using values from the `latitude` and `longitude` columns as [P48](https://database.factgrid.de/wiki/Property:P48).

In [ ]:
for index, row in prepared_df.iterrows():
    if (not pd.isna(row["latitude"])) and (not pd.isna(row["longitude"])):
        prepared_df.loc[index, "P48"] = f'@{row["latitude"]}/{row["longitude"]}'
prepared_df.drop_duplicates(subset="P48", inplace=True)
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde,Dde,Len,Den,P48
0,16698,46481629,11457,1250,1273.0,vor 1273,1782.0,NaN,NaN,12.368620,50.077752,Eger,11457,Online,"Klarissenkloster Eger (Cheb), Tschechien","""Gebäudekomplex Klarissenkloster Eger (Cheb), ...","""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862
1,1752,46479255,3349,1288,NaN,NaN,1416.0,NaN,NaN,18.221140,50.095780,Ratibor (Burgkapelle),3349,Online,"Kollegiatstift Ratibor (Racibórz), Polen","""Gebäudekomplex Kollegiatstift Ratibor (Racibó...","""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114
2,8714,14700,4502,1302,NaN,NaN,1533.0,NaN,NaN,13.315655,53.208969,NaN,4502,Online,Johanniterkommende Lychen,"""Gebäudekomplex Johanniterkommende Lychen""","""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655
3,13567,46483163,8372,1317,NaN,NaN,1603.0,1653.0,nach 1603,5.487612,51.960224,NaN,8372,Online,"Johanniterkommende Ingen, Niederlande","""Gebäudekomplex Johanniterkommende Ingen, Nied...","""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791
4,13569,46481587,8374,1327,NaN,NaN,1591.0,NaN,NaN,5.919402,52.554468,NaN,8374,Online,"Beginenhaus Kampen, Niederlande","""Gebäudekomplex Beginenhaus Kampen, Niederlande""","""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,5785,637,40089,1175,1225.0,vor 1225,1803.0,NaN,NaN,7.305031,50.516582,NaN,40089,Online,"Templerkommende, dann Johanniterkommende Hönni...","""Gebäudekomplex Templerkommende, dann Johannit...","""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308
108,16147,46484263,11227,1142,1143.0,1142/1143,1783.0,NaN,NaN,15.290278,49.960000,Sedletz,11227,Online,"Zisterzienserkloster Sedletz (Sedlec), Kutná H...","""Gebäudekomplex Zisterzienserkloster Sedletz (...","""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278
109,13704,46483164,8481,1447,1457.0,ca. 1452,1593.0,NaN,NaN,5.484776,52.223056,NaN,8481,Online,"Schwesterhaus St. Catharina, Nijkerk, Niederlande","""Gebäudekomplex Schwesterhaus St. Catharina, N...","""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399
110,15151,46483955,19937,1648,NaN,NaN,1810.0,NaN,NaN,15.586389,51.109483,Löwenberg,19937,Online,Franziskanerminoritenkloster Löwenberg (Lwówek...,"""Gebäudekomplex Franziskanerminoritenkloster L...","""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908


The prerequisite for connecting all building complexes with the locations in which they were found is that there are items in FactGrid for these locations. For the data collection in the monastery database, the open source service [geonames](https://www.geonames.org/) was the central tool. Therefore, there is a geonames ID in the monastery database for each location, which can be used to assign this location to a dataset in geonames. In FactGrid, there is also a qualifier (P418) for the GeoNames ID. This can be used to assign the location data to each other and to subsequently fill in missing locations. The notebook 1b - Place Matching describes this process.

### 1.4 Connection to places
Besides the coordinates, building complexes should also be connected with the places they are located in using the property [P83](https://database.factgrid.de/wiki/Property:P83). In order to achieve this, a matching between the table `gs_places` from the monastery database and FactGrid is needed. This can be achieved using a tool like OpenRefine, perform Coordinate Matching using QGis or match the names manually, if needed. 

**If you already have reconciled all the places** from the monasteries you want to upload against FactGrid, you can place the resulting table in `data/reconciliation/reconciliated_places.xlsx`. Make sure it has at least these two columns:
- `place_id`: the id from the gs_places table
- `factgrid_id`: The corresponding Q number from FactGrid
You can then continue with step [1.4.1](#section141)

**Otherwise** the following cell will create a table containing the place_id, the place name as well as some additional information that can be used for reconciling. It is stored at `data/reconciliation/reconcile_places.xlsx` Fill in the column `factgrid_id` using your preferred method and then place the resulting table at `data/reconciliation/reconciliated_places.xlsx`.

The cell below will do the following:
1. Load the tables gs_places
2. Merge it with the table monastery_locations_prepared to obtain all places that are relevant to the current dataset
3. Create a new table with columns that are relevant for reconciliation

In [ ]:
# 2. Merge with monastery locations
monastery_locations_places_merged = pd.merge(prepared_df, dataframes["gs_places"], how='left', left_on='place_id', right_on='id_places')
monastery_locations_places_merged
# 3. Create and save new table with relevant columns
reconcile_places = monastery_locations_places_merged[["place_id", "longitude_y", "latitude_y", "place_name", "gemeinde", "kreis", "geonames_id"]]
reconcile_places= reconcile_places.drop_duplicates(subset="place_id")
reconcile_places['factgrid_id'] = 0
reconcile_places.to_excel('data/reconciliation/reconcile_places.xlsx')
reconcile_places

,place_id,longitude_y,latitude_y,place_name,gemeinde,kreis,geonames_id,factgrid_id
0,46481629,12.370556,50.079444,Cheb (Eger),Cheb,Böhmen,3077835.0,0
1,46479255,18.219722,50.092222,Racibórz (Ratibor),NaN,NaN,3087584.0,0
2,14700,13.317167,53.207167,Lychen,Lychen,Uckermark,2874695.0,0
3,46483163,5.484720,51.959170,Ingen,NaN,NaN,2753316.0,0
4,46481587,5.900000,52.550000,Kampen,Kampen,Provinz Overijssel,6559054.0,0
...,...,...,...,...,...,...,...,...
96,637,7.305167,50.517333,Bad Hönningen,Bad Hönningen,Neuwied,2953434.0,0
97,46484263,15.268160,49.948390,Kutná Hora (Kuttenberg),NaN,NaN,3072463.0,0
98,46483164,5.486110,52.220000,Nijkerk,NaN,NaN,2750065.0,0
99,46483955,15.585820,51.110740,Lwówek Śląski (Löwenberg),NaN,Niederschlesien,3092638.0,0


<a id="section141"></a>
#### 1.4.1 Load reconciled places
Once the places have been reconciled, the filled table is loaded again and merged with the table from above. After this, **all entries that do not have a FactGrid ID are filtered out**. They are stored in a new file: `data/resulta/monasteries_without_factGrid.xlsx` so that they can be taken care of. 

The cell below will do the following:
1. Load the reconciled places
2. Merge them to the table with prepared monasteries
3. Filter out missing FactGrid Items and store them in a separate table

In [ ]:
# 1. Load the reconciled places
places_reconciled = pd.read_excel("data/reconciliation/reconciliated_places.xlsx")[["place_id", "factgrid_id"]]
# 2. Merge them to the table with prepared monasteries
prepared_df = pd.merge(prepared_df, places_reconciled, how="left", on="place_id")
prepared_df = prepared_df.rename(columns={"factgrid_id":"P83"})
prepared_df
# 3. Filter out missing FactGrid Items and store them in a separate table
missing_factgrid_ids = prepared_df[prepared_df['P83'].isna()]
missing_factgrid_ids.to_excel('data/results/monasteries_without_factGrid.xlsx')
prepared_df = prepared_df.dropna(subset = 'P83')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,location_name,id_gsn,status,monastery_name,Lde,Dde,Len,Den,P48,P83
0,16698,46481629,11457,1250,1273.0,vor 1273,1782.0,NaN,NaN,12.368620,...,Eger,11457,Online,"Klarissenkloster Eger (Cheb), Tschechien","""Gebäudekomplex Klarissenkloster Eger (Cheb), ...","""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862,Q102019
1,1752,46479255,3349,1288,NaN,NaN,1416.0,NaN,NaN,18.221140,...,Ratibor (Burgkapelle),3349,Online,"Kollegiatstift Ratibor (Racibórz), Polen","""Gebäudekomplex Kollegiatstift Ratibor (Racibó...","""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114,Q21722
2,8714,14700,4502,1302,NaN,NaN,1533.0,NaN,NaN,13.315655,...,NaN,4502,Online,Johanniterkommende Lychen,"""Gebäudekomplex Johanniterkommende Lychen""","""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655,Q88340
3,13567,46483163,8372,1317,NaN,NaN,1603.0,1653.0,nach 1603,5.487612,...,NaN,8372,Online,"Johanniterkommende Ingen, Niederlande","""Gebäudekomplex Johanniterkommende Ingen, Nied...","""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791,Q1347411
4,13569,46481587,8374,1327,NaN,NaN,1591.0,NaN,NaN,5.919402,...,NaN,8374,Online,"Beginenhaus Kampen, Niederlande","""Gebäudekomplex Beginenhaus Kampen, Niederlande""","""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705,Q87014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5785,637,40089,1175,1225.0,vor 1225,1803.0,NaN,NaN,7.305031,...,NaN,40089,Online,"Templerkommende, dann Johanniterkommende Hönni...","""Gebäudekomplex Templerkommende, dann Johannit...","""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308,Q82564
97,16147,46484263,11227,1142,1143.0,1142/1143,1783.0,NaN,NaN,15.290278,...,Sedletz,11227,Online,"Zisterzienserkloster Sedletz (Sedlec), Kutná H...","""Gebäudekomplex Zisterzienserkloster Sedletz (...","""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278,Q627082
98,13704,46483164,8481,1447,1457.0,ca. 1452,1593.0,NaN,NaN,5.484776,...,NaN,8481,Online,"Schwesterhaus St. Catharina, Nijkerk, Niederlande","""Gebäudekomplex Schwesterhaus St. Catharina, N...","""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399,Q1349336
99,15151,46483955,19937,1648,NaN,NaN,1810.0,NaN,NaN,15.586389,...,Löwenberg,19937,Online,Franziskanerminoritenkloster Löwenberg (Lwówek...,"""Gebäudekomplex Franziskanerminoritenkloster L...","""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908,Q80833


### 1.5 Create "instance of" statement
To state that these items are building complexes, the Item [Q635758](https://database.factgrid.de/wiki/Item:Q635758) (building complex) is connected to all entries using [P2](https://database.factgrid.de/wiki/Property:P2) (instance of)

In [ ]:
prepared_df["P2"] = "Q635758"
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,id_gsn,status,monastery_name,Lde,Dde,Len,Den,P48,P83,P2
0,16698,46481629,11457,1250,1273.0,vor 1273,1782.0,NaN,NaN,12.368620,...,11457,Online,"Klarissenkloster Eger (Cheb), Tschechien","""Gebäudekomplex Klarissenkloster Eger (Cheb), ...","""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862,Q102019,Q635758
1,1752,46479255,3349,1288,NaN,NaN,1416.0,NaN,NaN,18.221140,...,3349,Online,"Kollegiatstift Ratibor (Racibórz), Polen","""Gebäudekomplex Kollegiatstift Ratibor (Racibó...","""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114,Q21722,Q635758
2,8714,14700,4502,1302,NaN,NaN,1533.0,NaN,NaN,13.315655,...,4502,Online,Johanniterkommende Lychen,"""Gebäudekomplex Johanniterkommende Lychen""","""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655,Q88340,Q635758
3,13567,46483163,8372,1317,NaN,NaN,1603.0,1653.0,nach 1603,5.487612,...,8372,Online,"Johanniterkommende Ingen, Niederlande","""Gebäudekomplex Johanniterkommende Ingen, Nied...","""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791,Q1347411,Q635758
4,13569,46481587,8374,1327,NaN,NaN,1591.0,NaN,NaN,5.919402,...,8374,Online,"Beginenhaus Kampen, Niederlande","""Gebäudekomplex Beginenhaus Kampen, Niederlande""","""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705,Q87014,Q635758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5785,637,40089,1175,1225.0,vor 1225,1803.0,NaN,NaN,7.305031,...,40089,Online,"Templerkommende, dann Johanniterkommende Hönni...","""Gebäudekomplex Templerkommende, dann Johannit...","""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308,Q82564,Q635758
97,16147,46484263,11227,1142,1143.0,1142/1143,1783.0,NaN,NaN,15.290278,...,11227,Online,"Zisterzienserkloster Sedletz (Sedlec), Kutná H...","""Gebäudekomplex Zisterzienserkloster Sedletz (...","""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278,Q627082,Q635758
98,13704,46483164,8481,1447,1457.0,ca. 1452,1593.0,NaN,NaN,5.484776,...,8481,Online,"Schwesterhaus St. Catharina, Nijkerk, Niederlande","""Gebäudekomplex Schwesterhaus St. Catharina, N...","""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399,Q1349336,Q635758
99,15151,46483955,19937,1648,NaN,NaN,1810.0,NaN,NaN,15.586389,...,19937,Online,Franziskanerminoritenkloster Löwenberg (Lwówek...,"""Gebäudekomplex Franziskanerminoritenkloster L...","""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908,Q80833,Q635758


### 1.6 GS Vocabulary Statement
In order to keep a mapping between the monastery database and FactGrid, every item will receive a distinct vocabulary term that is constructed using the `id_monastery_location` from the monastery database. The FactGrid Property to use is [P1301](https://database.factgrid.de/wiki/Property:P1301) (GS vocabulary term).

Vocabulary terms for building complex items are constructed as follows:

`GSMonasteryLocation<id_monastery_location>`

The cell below will create a new column for P3101 and fill it using the construction template above.


In [ ]:
prepared_df['P1301'] = prepared_df['id_monastery_location'].apply(lambda x: f'\"GSMonasteryLocation{x}\"')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,status,monastery_name,Lde,Dde,Len,Den,P48,P83,P2,P1301
0,16698,46481629,11457,1250,1273.0,vor 1273,1782.0,NaN,NaN,12.368620,...,Online,"Klarissenkloster Eger (Cheb), Tschechien","""Gebäudekomplex Klarissenkloster Eger (Cheb), ...","""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862,Q102019,Q635758,"""GSMonasteryLocation16698"""
1,1752,46479255,3349,1288,NaN,NaN,1416.0,NaN,NaN,18.221140,...,Online,"Kollegiatstift Ratibor (Racibórz), Polen","""Gebäudekomplex Kollegiatstift Ratibor (Racibó...","""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114,Q21722,Q635758,"""GSMonasteryLocation1752"""
2,8714,14700,4502,1302,NaN,NaN,1533.0,NaN,NaN,13.315655,...,Online,Johanniterkommende Lychen,"""Gebäudekomplex Johanniterkommende Lychen""","""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655,Q88340,Q635758,"""GSMonasteryLocation8714"""
3,13567,46483163,8372,1317,NaN,NaN,1603.0,1653.0,nach 1603,5.487612,...,Online,"Johanniterkommende Ingen, Niederlande","""Gebäudekomplex Johanniterkommende Ingen, Nied...","""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791,Q1347411,Q635758,"""GSMonasteryLocation13567"""
4,13569,46481587,8374,1327,NaN,NaN,1591.0,NaN,NaN,5.919402,...,Online,"Beginenhaus Kampen, Niederlande","""Gebäudekomplex Beginenhaus Kampen, Niederlande""","""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705,Q87014,Q635758,"""GSMonasteryLocation13569"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5785,637,40089,1175,1225.0,vor 1225,1803.0,NaN,NaN,7.305031,...,Online,"Templerkommende, dann Johanniterkommende Hönni...","""Gebäudekomplex Templerkommende, dann Johannit...","""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308,Q82564,Q635758,"""GSMonasteryLocation5785"""
97,16147,46484263,11227,1142,1143.0,1142/1143,1783.0,NaN,NaN,15.290278,...,Online,"Zisterzienserkloster Sedletz (Sedlec), Kutná H...","""Gebäudekomplex Zisterzienserkloster Sedletz (...","""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278,Q627082,Q635758,"""GSMonasteryLocation16147"""
98,13704,46483164,8481,1447,1457.0,ca. 1452,1593.0,NaN,NaN,5.484776,...,Online,"Schwesterhaus St. Catharina, Nijkerk, Niederlande","""Gebäudekomplex Schwesterhaus St. Catharina, N...","""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399,Q1349336,Q635758,"""GSMonasteryLocation13704"""
99,15151,46483955,19937,1648,NaN,NaN,1810.0,NaN,NaN,15.586389,...,Online,Franziskanerminoritenkloster Löwenberg (Lwówek...,"""Gebäudekomplex Franziskanerminoritenkloster L...","""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908,Q80833,Q635758,"""GSMonasteryLocation15151"""


### 1.8 Source Statements

Every Statement in FactGrid should be supported by a Source/Reference. To achieve this, a source column `S471` is added after each relevant property to link to the Monastery Database Entries using the Property [P471](https://database.factgrid.de/wiki/Property:P471).

In [ ]:
final_table = prepared_df.copy()
for colname in ["P48", "P83"]:
    final_table.insert(final_table.columns.get_loc(colname)+1, "S471", final_table["gsn_id"].apply(lambda x:f'\"{x}\"'), allow_duplicates=True)
final_table["P131"] = "Q153178"
final_table

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,Dde,Len,Den,P48,S471,P83,S471,P2,P1301,P131
0,16698,46481629,11457,1250,1273.0,vor 1273,1782.0,NaN,NaN,12.368620,...,"""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862,"""11457""",Q102019,"""11457""",Q635758,"""GSMonasteryLocation16698""",Q153178
1,1752,46479255,3349,1288,NaN,NaN,1416.0,NaN,NaN,18.221140,...,"""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114,"""3349""",Q21722,"""3349""",Q635758,"""GSMonasteryLocation1752""",Q153178
2,8714,14700,4502,1302,NaN,NaN,1533.0,NaN,NaN,13.315655,...,"""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655,"""4502""",Q88340,"""4502""",Q635758,"""GSMonasteryLocation8714""",Q153178
3,13567,46483163,8372,1317,NaN,NaN,1603.0,1653.0,nach 1603,5.487612,...,"""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791,"""8372""",Q1347411,"""8372""",Q635758,"""GSMonasteryLocation13567""",Q153178
4,13569,46481587,8374,1327,NaN,NaN,1591.0,NaN,NaN,5.919402,...,"""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705,"""8374""",Q87014,"""8374""",Q635758,"""GSMonasteryLocation13569""",Q153178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5785,637,40089,1175,1225.0,vor 1225,1803.0,NaN,NaN,7.305031,...,"""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308,"""40089""",Q82564,"""40089""",Q635758,"""GSMonasteryLocation5785""",Q153178
97,16147,46484263,11227,1142,1143.0,1142/1143,1783.0,NaN,NaN,15.290278,...,"""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278,"""11227""",Q627082,"""11227""",Q635758,"""GSMonasteryLocation16147""",Q153178
98,13704,46483164,8481,1447,1457.0,ca. 1452,1593.0,NaN,NaN,5.484776,...,"""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399,"""8481""",Q1349336,"""8481""",Q635758,"""GSMonasteryLocation13704""",Q153178
99,15151,46483955,19937,1648,NaN,NaN,1810.0,NaN,NaN,15.586389,...,"""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908,"""19937""",Q80833,"""19937""",Q635758,"""GSMonasteryLocation15151""",Q153178


### 1.9 Finalize Table

As a last step, all columns that are not needed for import are dropped, an empty qid column is added, so that new items will be created and the resulting table is stored under:
- `data/results/import_building_complexes.xlsx`
- `data/results/import_building_complexes.csv`

In [ ]:
def df_to_qs_v1(df:pd.DataFrame):
    cols=pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns=cols
    print(df)
    df_dict = df.to_dict(orient="records")
    output_collect = ""
    for item in df_dict:
        output = ""
        create_flag = False
        keys = list(item.keys())
        i = 0
        qid = None
        while i < len(keys):
            out_str = ""
            k = keys[i]
            if k.startswith("S") or k.startswith("qal"):
                i += 1
                continue
            if k == "qid":
                if pd.isna(item["qid"]):
                    create_flag = True
                    out_str += "CREATE\n"
                else:
                    qid = item["qid"]
            elif not pd.isna(item[k]):
                if create_flag:
                    out_str += "LAST"
                else:
                    out_str += qid
                out_str += f"\t{k}\t{item[k]}"
                while i+1 < len(keys) and (keys[i+1].startswith("S") or keys[i+1].startswith("qal")):
                    k = keys[i+1]
                    if k.startswith("qal"):
                        k_rep = k.replace("qal", "P")
                        if(not pd.isna(item[k])):
                            out_str += f'\t{k_rep.split(".")[0]}\t{item[k]}'
                        i += 1
                    else:
                        if(not pd.isna(item[k])):
                            out_str += f'\t{k.split(".")[0]}\t{item[k]}'
                        i += 1
                out_str += "\n"
            output += out_str
            i += 1
        output_collect += output
    return output_collect
                
                



In [ ]:
final_table = final_table.drop(columns=["id_monastery_location", "place_id", "gsn_id", "location_begin_tpq", "location_begin_taq", "location_begin_note", "location_end_tpq", "location_end_taq", "location_end_note", "longitude", "latitude", "location_name", "id_gsn", "status", "monastery_name"])
final_table.insert(0, "qid", np.nan)
final_table.to_excel("data/results/import_building_complexes.xlsx", index=False)
final_table.to_csv("data/results/import_building_complexes.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§")
with open("data/results/import_building_complexes.tsv", "w") as file:
    file.write(df_to_qs_v1(final_table))
final_table

     qid                                                Lde  \
0    NaN  "Gebäudekomplex Klarissenkloster Eger (Cheb), ...   
1    NaN  "Gebäudekomplex Kollegiatstift Ratibor (Racibó...   
2    NaN         "Gebäudekomplex Johanniterkommende Lychen"   
3    NaN  "Gebäudekomplex Johanniterkommende Ingen, Nied...   
4    NaN   "Gebäudekomplex Beginenhaus Kampen, Niederlande"   
..   ...                                                ...   
96   NaN  "Gebäudekomplex Templerkommende, dann Johannit...   
97   NaN  "Gebäudekomplex Zisterzienserkloster Sedletz (...   
98   NaN  "Gebäudekomplex Schwesterhaus St. Catharina, N...   
99   NaN  "Gebäudekomplex Franziskanerminoritenkloster L...   
100  NaN  "Gebäudekomplex Benediktinerpropstei Sankt Kat...   

                                                   Dde  \
0    "Gebäudekomplex Eger des Klarissenklosters Ege...   
1    "Gebäudekomplex Ratibor (Burgkapelle) des Koll...   
2       "Gebäudekomplex der Johanniterkommende Lychen"   
3    "Gebäu

,qid,Lde,Dde,Len,Den,P48,S471,P83,S471.1,P2,P1301,P131
0,NaN,"""Gebäudekomplex Klarissenkloster Eger (Cheb), ...","""Gebäudekomplex Eger des Klarissenklosters Ege...","""Building complex Benedicts of St. Michael, Me...","""Building complex of the Benedicts of St. Mich...",@50.077752/12.36862,"""11457""",Q102019,"""11457""",Q635758,"""GSMonasteryLocation16698""",Q153178
1,NaN,"""Gebäudekomplex Kollegiatstift Ratibor (Racibó...","""Gebäudekomplex Ratibor (Burgkapelle) des Koll...","""Building complex Benedictine monastery Vornba...","""Building complex Vornbach of the Benedictine ...",@50.09578/18.22114,"""3349""",Q21722,"""3349""",Q635758,"""GSMonasteryLocation1752""",Q153178
2,NaN,"""Gebäudekomplex Johanniterkommende Lychen""","""Gebäudekomplex der Johanniterkommende Lychen""","""Building complex Cistercian nunnery Ophoven (...","""Building complex Ophoven of the Cistercian nu...",@53.208969/13.315655,"""4502""",Q88340,"""4502""",Q635758,"""GSMonasteryLocation8714""",Q153178
3,NaN,"""Gebäudekomplex Johanniterkommende Ingen, Nied...","""Gebäudekomplex der Johanniterkommende Ingen, ...","""Building complex Capuchin friary Karlstadt""","""Building complex of the Capuchin friary Karls...",@51.9602244312605/5.48761203580791,"""8372""",Q1347411,"""8372""",Q635758,"""GSMonasteryLocation13567""",Q153178
4,NaN,"""Gebäudekomplex Beginenhaus Kampen, Niederlande""","""Gebäudekomplex des Beginenhauses Kampen, Nied...","""Building complex Benedictine monastery of Bas...","""Building complex of the Benedictine monastery...",@52.5544680656827/5.91940200615705,"""8374""",Q87014,"""8374""",Q635758,"""GSMonasteryLocation13569""",Q153178
...,...,...,...,...,...,...,...,...,...,...,...,...
96,NaN,"""Gebäudekomplex Templerkommende, dann Johannit...","""Gebäudekomplex der Templerkommende, dann Joha...",NaN,NaN,@50.51658173801344/7.305030721604308,"""40089""",Q82564,"""40089""",Q635758,"""GSMonasteryLocation5785""",Q153178
97,NaN,"""Gebäudekomplex Zisterzienserkloster Sedletz (...","""Gebäudekomplex Sedletz des Zisterzienserklost...",NaN,NaN,@49.96/15.290278,"""11227""",Q627082,"""11227""",Q635758,"""GSMonasteryLocation16147""",Q153178
98,NaN,"""Gebäudekomplex Schwesterhaus St. Catharina, N...","""Gebäudekomplex des Schwesterhauses St. Cathar...",NaN,NaN,@52.2230564143223/5.48477573772399,"""8481""",Q1349336,"""8481""",Q635758,"""GSMonasteryLocation13704""",Q153178
99,NaN,"""Gebäudekomplex Franziskanerminoritenkloster L...","""Gebäudekomplex Löwenberg des Franziskanermino...",NaN,NaN,@51.10948301890577/15.586388627441908,"""19937""",Q80833,"""19937""",Q635758,"""GSMonasteryLocation15151""",Q153178


In [ ]:
print(df_to_qs_v1(final_table))

     qid                                                Lde  \
0    NaN  "Gebäudekomplex Klarissenkloster Eger (Cheb), ...   
1    NaN  "Gebäudekomplex Kollegiatstift Ratibor (Racibó...   
2    NaN         "Gebäudekomplex Johanniterkommende Lychen"   
3    NaN  "Gebäudekomplex Johanniterkommende Ingen, Nied...   
4    NaN   "Gebäudekomplex Beginenhaus Kampen, Niederlande"   
..   ...                                                ...   
96   NaN  "Gebäudekomplex Templerkommende, dann Johannit...   
97   NaN  "Gebäudekomplex Zisterzienserkloster Sedletz (...   
98   NaN  "Gebäudekomplex Schwesterhaus St. Catharina, N...   
99   NaN  "Gebäudekomplex Franziskanerminoritenkloster L...   
100  NaN  "Gebäudekomplex Benediktinerpropstei Sankt Kat...   

                                                   Dde  \
0    "Gebäudekomplex Eger des Klarissenklosters Ege...   
1    "Gebäudekomplex Ratibor (Burgkapelle) des Koll...   
2       "Gebäudekomplex der Johanniterkommende Lychen"   
3    "Gebäu

**Once you uploaded the building complexes** make sure to keep a table that maps the newly created Q-numbers to the GS Vocabulary Term (`P1301`). Those will be needed in step 3.